## Книжная лихорадка

***Предыстория***   
Коронавирус застал мир врасплох, изменив привычный порядок вещей.   
В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры.   
Зато стало больше времени для книг.   
Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. 
Моя компания решила быть на волне и купила крупный сервис для чтения книг по подписке.  
 
**Задача** максимум в этом суровом мире корановируса — ~~выжить~~ проанализировать базу данных книжного сервиса.   
В ней информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.   
Эти данные помогут сформулировать ценностное предложение для нового продукта.   
 
**Описание данных**   
  
**Таблица `books`** (<u>содержит данные о книгах</u>):   
- `book_id` — идентификатор книги   
- `author_id` — идентификатор автора   
- `title` — название книги   
- `num_pages` — количество страниц   
- `publication_date` — дата публикации книги   
- `publisher_id` — идентификатор издателя   
   
**Таблица `authors`** (<u>содержит данные об авторах</u>):   
- `author_id` — идентификатор автора 
- `author` — имя автора   
   
**Таблица `publishers`** (<u>содержит данные об издательствах</u>):   
- `publisher_id` — идентификатор издательства 
- `publisher` — название издательства   
   
**Таблица `ratings`** (<u>содержит данные о пользовательских оценках книг</u>):   
- `rating_id` — идентификатор оценки  
- `book_id` — идентификатор книги  
- `username` — имя пользователя, оставившего оценку  
- `rating` — оценка книги   
   
**Таблица `reviews`** (<u>содержит данные о пользовательских обзорах</u>):   
- `review_id` — идентификатор обзора  
- `book_id` — идентификатор книги  
- `username` — имя автора обзора  
- `text` — текст обзора


In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp', 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Перед каждым решением задачи буду выводить таблицу с помощью SQL запроса, чтобы посмотреть на данные - так удобнее ориентироваться.

## Посчитаю, сколько книг вышло после 1 января 2000 года

In [4]:
query = '''
SELECT * 
FROM books 
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
SELECT COUNT(DISTINCT book_id)
FROM books 
WHERE publication_date > '2000-01-01'
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 уникальных единиц книжной продукции.

## Для каждой книги посчитаю количество обзоров и среднюю оценку

In [6]:
query = '''
SELECT * 
FROM ratings
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''
SELECT * 
FROM reviews
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Сджоиним три таблицы по `book_id`.

In [8]:
query = '''
SELECT b.book_id,
       COUNT(DISTINCT rev.review_id) AS count_reviews,        
       AVG(rat.rating) AS avg_rat
FROM books AS b
LEFT JOIN reviews AS rev ON b.book_id = rev.book_id 
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY b.book_id
ORDER BY avg_rat
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,count_reviews,avg_rat
0,303,2,1.50
1,371,2,2.00
2,316,2,2.00
3,202,3,2.00
4,915,3,2.25
...,...,...,...
995,421,2,5.00
996,418,2,5.00
997,169,2,5.00
998,901,2,5.00


Мы имеем 1 000 уникальных единиц книжной продукции.  
- **MIN рейтинг:** 1.50  
- **MAX рейтинг:** 5.00     

Также установлено, что количество отзывов с рейтингом не коррелирует.

## Определю издательство, которое выпустило наибольшее число книг толще 50 страниц  
Так исключу из анализа брошюры

In [9]:
query = '''
SELECT * 
FROM publishers
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Буду джоинить две таблицы по `publisher_id`. Сначала выведу отдельно во временную таблицу книжную продукцию с более 50-ти страницами, чтобы избавиться от брошюр. После сджойню с таблицей `publishers` и выведу нужные мне данные.

In [10]:
query = '''
WITH 
p_50 AS (
SELECT *
FROM books
WHERE num_pages > 50)

SELECT pub.publisher,
       COUNT(p_50.book_id),       
       MAX(COUNT(p_50.book_id)) OVER () AS max_ct
FROM p_50
INNER JOIN publishers AS pub ON p_50.publisher_id = pub.publisher_id
GROUP BY pub.publisher
ORDER  BY COUNT(p_50.book_id) DESC
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count,max_ct
0,Penguin Books,42,42
1,Vintage,31,42
2,Grand Central Publishing,25,42
3,Penguin Classics,24,42
4,Ballantine Books,19,42
...,...,...,...
329,Turtleback,1,42
330,Atheneum Books for Young Readers: Richard Jack...,1,42
331,Penguin Signet,1,42
332,Victor Gollancz,1,42


Книжная продукция с более 50-ти страницами выпускается 334 издательствами.  
Наибольшее число книжной продукции с более 50-ти страницами выпущено издательством **Penguin Books** в количестве 42.

## Определю автора с самой высокой средней оценкой книг  
Учиту только книги с 50 и более оценками

In [11]:
query = '''
SELECT * 
FROM authors
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Буду джоинить три таблицы по `author_id` и `book_id`. Сначала выведу отдельно во временную таблицу книжную продукцию с более 50-ти оценками. После произведу джоины таблиц `authors`, `ratings`, `books` и выведу нужные мне данные.

In [12]:
query = '''
WITH 
r_50 AS (
SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50)

SELECT a.author_id, 
       a.author, 
       AVG(rat.rating) AS avg_rating,
       MAX(AVG(rat.rating)) OVER () AS max_avg_r
FROM books AS b
INNER JOIN r_50 ON b.book_id = r_50.book_id
INNER JOIN authors AS a ON b.author_id = a.author_id
INNER JOIN ratings AS rat ON r_50.book_id = rat.book_id
GROUP BY a.author_id, a.author
ORDER BY AVG(rat.rating) DESC
LIMIT 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg_rating,max_avg_r
0,236,J.K. Rowling/Mary GrandPré,4.287097,4.287097


Самая высокая средняя оценка присвоена авторам с идентификатором 236 **J.K. Rowling/Mary GrandPré**. Их рейтинг составляет 4.29.

## Посчитаю среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [13]:
query = '''
SELECT * 
FROM ratings
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [14]:
query = '''
SELECT * 
FROM reviews
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Буду джоинить две таблицы `ratings` и `reviews` по `username` и выведу нужные мне данные.

In [15]:
query = '''
SELECT COUNT(DISTINCT rev.text)/ COUNT(DISTINCT rev.username) AS avg_rv_count
FROM ratings AS rat
INNER JOIN reviews AS rev
ON rat.username = rev.username
WHERE rat.username IN ( 
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48)
'''

pd.io.sql.read_sql(query, con = engine)

,avg_rv_count
0,24


Среднее количество обзоров от пользователей, поставивших более 48 оценок, равно 24.

## Общие выводы

После 1 января 2000 года вышло 819 уникальных единиц книжной продукции.  
  
Мы имеем 1 000 уникальных единиц книжной продукции.  
- **MIN рейтинг:** 1.50  
- **MAX рейтинг:** 5.00     

Также установлено, что количество отзывов с рейтингом не коррелирует.  

Книжная продукция с более 50-ти страницами выпускается 334 издательствами.  
Наибольшее число книжной продукции с более 50-ти страницами выпущено издательством **Penguin Books** в количестве 42.  

Самая высокая средняя оценка присвоена авторам с идентификатором 236 **J.K. Rowling/Mary GrandPré**. Их рейтинг составляет 4.29.  

Среднее количество обзоров от пользователей, поставивших более 48 оценок, равно 24.